In [1]:
import pandas as pd
import joblib
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
import torch
import os
import torch.optim as optim
import warnings 
warnings.simplefilter('ignore')
from torch.nn import Linear,ReLU,Sigmoid
from utils import *
torch.cuda.is_available()

True

In [2]:
os.listdir('./data')

['train_preprocessed',
 'tag盤點表20200323.xlsx',
 'pred_0997_all.csv',
 'tag盤點表_cleaned.csv',
 'train_cleaned.csv',
 'PA_201605-2012001更新R7(20200323).xlsx']

# load pretrained model

In [3]:
critic = joblib.load('./model/net.pkl')
critic = critic.eval()
mm_x = joblib.load('./model/mm_x.pkl')
mm_y = joblib.load('./model/mm_y.pkl')

In [4]:
data = joblib.load('./data/train_preprocessed')

In [5]:
x_cols = data['x_col']
y_cols = data['y_col']

In [6]:
tag = pd.read_csv('./data/tag盤點表_cleaned.csv')
tag_map = dict(zip(tag['TAG'],tag['chinese']))

# API

In [7]:
'''
input: 0997 value
output: control advice
'''
class API(object):
    def __init__(self,critic,mm_x,mm_y,x_cols,y_cols,tag_map):
        self.critic = critic
        self.mm_x = mm_x
        self.mm_y = mm_y
        self.x_cols = x_cols
        self.y_cols = y_cols
        # Generator
        self.generator = self._build_generator(1,37*14).double().cuda()
        self.generator.apply(self._init_weights)
        # bound
        self.sc_min = mm_y.data_min_[0]
        self.sc_max = mm_y.data_max_[0]
        # target
        self.target = None
        # noise_iter and optimizer
        self.noise_iter = self._build_noise_iter()
        self.optimizer = optim.Adam(self.generator.parameters(),lr=1e-4)
        # obj
        self.obj_function = self._obj_function
        # tag_map
        self.tag_map = tag_map
        # noise
        self.noise = torch.tensor(np.random.normal(loc=0,scale=1,size=(5000,1))).cuda()
    
    def get_advice(self,target):
        # set user want target value
        self.target = target
        
        # train the generator close to target 
        self.generator = self._train(self.generator,self.noise_iter,self.obj_function,
                                     self.optimizer,num_epochs=100)
        # after training get output
        output = self.critic(self.generator(self.noise)).detach().cpu().numpy()
        output = self.mm_y.inverse_transform(output)
        output = pd.DataFrame(output,columns=['0997'])
        
        # get best result base on output
        best_idx = np.argmin((output-self.target).values)
        result = pd.DataFrame(self.generator(self.noise).detach().cpu().numpy())
        result[:] = self.mm_x.inverse_transform(result)
        r1 = result.iloc[best_idx,:].values.reshape(37,14)
        r1 = pd.DataFrame(r1,columns=self.x_cols)
        
        # calculate best result's median max and min
        r1.loc['median'] = r1.median(axis=0)
        r1.loc['max'] = r1.max(axis=0)
        r1.loc['min'] = r1.min(axis=0)
        
        # add chinese tag
        r1 = r1.iloc[-3:].T
        r1['chinese'] = r1.index.map(self.tag_map)
        r1 = r1[['chinese','median','max','min']]
        
        # return control advice and 0997_value
        return r1,output.loc[[best_idx]]
    
    @staticmethod
    def _build_generator(input_shape,output_shape):
        net = torch.nn.Sequential(
            Linear(input_shape,256),ReLU(),
            Linear(256,output_shape),Sigmoid())
        return net
    
    @staticmethod
    def _init_weights(m):
        if hasattr(m,'weight'):
            torch.nn.init.xavier_uniform(m.weight)
        if hasattr(m,'bias'):  
            m.bias.data.fill_(0)
    
    def _obj_function(self,x):
        y = self.critic(x)
        y = (self.sc_max-self.sc_min)*y + self.sc_min
        loss = ((y-self.target)**2)**(0.5)
        return loss.mean()
    
    @staticmethod
    def _build_noise_iter():
        noise = torch.tensor(np.random.normal(loc=0,scale=1,size=(5000,1))).cuda()
        noise_datasets = torch.utils.data.TensorDataset(noise)
        noise_iter = torch.utils.data.DataLoader(noise_datasets,batch_size=256)
        return noise_iter
    
    @staticmethod
    def _train(net,noise_iter,obj_function,optimizer,num_epochs=100):
        train_history = []
        for epoch in tqdm(range(num_epochs)):
            train_avg_loss = 0
            t = 0
            for noise in noise_iter:
                noise = noise[0]
                z = net(noise).cuda()
                loss = obj_function(z)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                train_avg_loss += loss.item()
                t += 1
            train_avg_loss /= t
            #print("epochs {} loss {:.4f}".format(epoch,train_avg_loss))
            train_history.append(train_avg_loss)
        # plt train loss
        #plt.plot(np.array(train_history))
        #plt.title('train loss')
        # return trained net
        return net

In [8]:
A = API(critic,mm_x,mm_y,x_cols,y_cols,tag_map)

In [9]:
advice,value = A.get_advice(7000)

In [10]:
advice

,chinese,median,max,min
MLPAP_FQ-0619.PV,OX入料量,6602.686979,6880.164993,5903.049214
MLPAP_FR-0632.PV,汽機風量,79.663580,80.641300,63.847267
MLPAP_LIC-0738.PV,汽鼓(V211)鍋爐水液位,35.970720,36.286459,34.740894
MLPAP_LRC-0729.PV,鍋爐水液位控制,44.928308,47.118155,41.459878
MLPAP_PIC-0761.PV,鹽浴冷卻器(E112)出口壓力,49.450470,49.490387,49.420309
MLPAP_TJ-0757B.PV,反應槽鹽浴溫度,354.346206,355.905518,353.090225
MLPAP_TJC-0627.PV,OX預熱器出口溫度,149.423875,150.749087,148.399281
MLPAP_TJC-0757.PV,反應器A/B側鹽浴溫度差,0.554237,0.988008,0.203618
MLPAP_TJC-0757A.PV,反應槽鹽浴溫度,353.412876,354.943897,352.266139
MLPAP_TJC-1721.PV,轉換式冷凝器冷油溫度,60.804608,61.928917,59.957190


In [11]:
value

,0997
1167,6998.286363
